In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import pathlib
import random
import pandas as pd
import sys

from credentials import CONNECTION_INFO
from constants import *

import encoders
import db_connect
import helpers

# Callbacks for use with TensorFlow
from tf_helpers import modulo_output, modulo_distance_loss, modulo_distance_accuracy, modulo_rounded_accuracy, initialize_save_best

2024-11-30 21:32:58.279224: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-30 21:32:58.343612: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-30 21:32:58.359547: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-30 21:32:58.364056: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-30 21:32:58.415948: I tensorflow/core/platform/cpu_feature_guar

## Config
This notebook has a lot of options to adjust, most of which are controlled here.

In [3]:
ENCODER = encoders.ENCODER_CAESAR
CHUNK_SIZE = 128 # Was 512
PROCESSING_UNITS = CHUNK_SIZE // 4

EXTRA_CHECKS = True # Whether to run some (potentially slow) debug checks

INFER_TEXT = False # Text inferrence isn't really working. I don't know how to combine the time-distributed results.
INFER_KEY = not INFER_TEXT

USE_CUSTOM_METRICS = True

if INFER_TEXT:
    MAIN_ACCURACY_METRIC = "mae"
    LOSS_METRIC = "mean_squared_error"
    OUTPUT_SIZE = CHUNK_SIZE
    OPTIMIZER = "sgd"
else:
    MAIN_ACCURACY_METRIC = "mae"
    LOSS_METRIC = "mae"
    OPTIMIZER = "adamax"    

    if ENCODER == encoders.ENCODER_CAESAR:
        OUTPUT_SIZE = 1
    elif ENCODER == encoders.ENCODER_SUBST:
        OUTPUT_SIZE = len(encoders.CHARSET)
    else:
        raise Exception(f"Unsupported encoder {ENCODER}")

ENCRYPTED_FILE_LIMIT = -1 # -1 to disable limit

BASE_TRAIN_PCT = 0.75   # Start here. If train or test count would exceed the max, reduce it. Note 0.75 is the default.
MAX_TRAIN_COUNT = -1 # -1 to disable; some setups start running out of memory around 100K
MAX_TEST_COUNT =  -1 # -1 to disable
SPLIT_SEED = 42

LOAD_BEST_MODEL = False # If False, a new model will be created from scratch
SAVE_BEST_MODEL = True
BEST_PATH = './saved_models/best.keras'

# Whether to run the tuner or the hard-coded network build code
TUNE_NETWORK = False
TUNE_QUICKLY = False # Set True to sanity check the model builder
BUILD_NETWORK = not TUNE_NETWORK
TRAIN_MODEL = BUILD_NETWORK and not LOAD_BEST_MODEL

TUNER_DIRECTORY = "tuner_projects"
TUNER_PROJECT_NAME = "KT"

EPOCHS = 10
BATCH_SIZE = int(max(32, round(256 * (512/CHUNK_SIZE)))) # Default is 32 -- going higher speeds things up a LOT, but may cause memory problems
SCALE = True

CHUNK_SIZE, PROCESSING_UNITS, BATCH_SIZE, OUTPUT_SIZE

(128, 32, 1024, 1)

# Data Retrieval and Structuring

In [4]:
db = db_connect.DB(CONNECTION_INFO)

with db.get_session() as session:
    # Get database IDs for encoders and key types
    (encoder_ids, key_type_id) = db.get_id_maps(session)

    # Map source ID to plaintext file (1) details, and source ID to corresponding ciphertext files (1+) details
    (sid_to_p, sid_to_c) = db.get_source_maps(session, ENCRYPTED_FILE_LIMIT, encoder_ids[ENCODER], test_only=False)

len(sid_to_p), len(sid_to_c)

(35, 35)

In [5]:
# Build up the features (X, the cipher texts as offsets) and targets (y, either the plain texts as offsets OR the key).
# Note targets are not necessarily unique.
X = []
y = []

with db.get_session() as session:
    for sid in sid_to_p:
        if INFER_TEXT:
            plaintext = encoders.string_to_offsets(helpers.read_text_file(sid_to_p[sid].path))
            target_chunks = helpers.chunkify(plaintext, CHUNK_SIZE)    
    
        for c in sid_to_c[sid]:
            ciphertext = encoders.string_to_offsets(helpers.read_text_file(c.path))
            feature_chunks = helpers.chunkify(ciphertext, CHUNK_SIZE)

            if INFER_KEY:                
                if ENCODER == encoders.ENCODER_CAESAR:
                    key_value = float(db.get_key_by_id(session, c.key_id).value)

                    if EXTRA_CHECKS:
                        # Decode with the key we got from the DB, make sure it actually works
                        CHECK_CHANCE = 0.1
                        if random.random() < CHECK_CHANCE:
                            plaintext = encoders.string_to_offsets(helpers.read_text_file(sid_to_p[sid].path))
                            plainttext_str = encoders.offsets_to_string(plaintext)
                            ciphertext_str = encoders.offsets_to_string(ciphertext)
                            decoded_str = encoders.decode_caesar(ciphertext_str, int(key_value))
                            if decoded_str != plainttext_str:                                                    
                                print(decoded_str == plainttext_str)
                                print(decoded_str[0:128], plainttext_str[0:128])
                                raise Exception("Decode error")
                    
                elif ENCODER == encoders.ENCODER_SUBST:
                    key_str = db.get_key_by_id(session, c.key_id).value
                    key_value_ints = encoders.string_to_offsets(key_str)
                    key_value = np.array(key_value_ints).astype(float)

                else:
                    raise Exception(f"Unsupported encoder {ENCODER}")
        
            for i in range (len(feature_chunks)):
                X.append(np.array(feature_chunks[i]).astype(float))

                if INFER_TEXT:
                    y.append(np.array(target_chunks[i]).astype(float))

                if INFER_KEY:
                    y.append(key_value)

X = np.array(X)
y = np.array(y)

X.shape, y.shape, sys.getsizeof(X), sys.getsizeof(y)

((383568, 128), (383568,), 392773760, 3068656)

In [6]:
# Split the preprocessed data into a training and testing dataset
# Note we have excluded "test_only" files above, they will be used for later validation.

train_count = int(round(len(y) * BASE_TRAIN_PCT))
if train_count > MAX_TRAIN_COUNT and MAX_TRAIN_COUNT > -1:
    print(f"Train count would be {train_count}")
    train_count = int(MAX_TRAIN_COUNT)
print(f"Train count is {train_count}")

test_count = len(y) - train_count
if test_count > MAX_TEST_COUNT and MAX_TEST_COUNT > -1:
    print(f"Test count would be {test_count}")
    test_count = int(MAX_TEST_COUNT)
print(f"Test count is {test_count}")

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_count, test_size=test_count, random_state=SPLIT_SEED)

# The pre-split data sets are no longer needed, and take up a lot of memory, so get rid of them
del X
del y

Train count is 287676
Test count is 95892


In [7]:
if SCALE:
    # Create a StandardScaler instances
    scaler = StandardScaler()
    
    # Fit the StandardScaler
    X_scaler = scaler.fit(X_train)
    
    # Scale the data
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)    
else:
    # Leave the inputs unscaled
    X_train_scaled = X_train
    X_test_scaled = X_test

to_show = min(16, CHUNK_SIZE)
X_train_scaled.shape, X_test_scaled.shape, X_train_scaled[0][0:to_show], X_test_scaled[0][0:to_show]

((287676, 128),
 (95892, 128),
 array([-1.62100853, -0.20988748, -0.56571487,  1.5519478 ,  0.14124982,
        -0.91938531, -1.62073862, -1.26721369, -0.56513113, -0.91645448,
        -0.56262348,  1.20045693, -0.56348523, -0.91687292,  0.14046587,
         0.14047778]),
 array([-0.20992427,  0.49578658,  0.14028845, -0.91727933,  0.49420995,
         1.19967399, -0.91536893,  0.14335206,  0.49365611, -1.62284044,
         0.84967484, -0.91674187,  1.20019793, -1.62255094,  0.49315509,
         0.14047778]))

In [8]:
# Reshape the data as required for the model

print(f"Original shapes: {X_train.shape}, {X_test.shape}, {y_train.shape}, {y_test.shape}")

X_train = X_train.reshape((-1, CHUNK_SIZE, 1)) 
X_train_scaled = X_train_scaled.reshape((-1, CHUNK_SIZE, 1)) 
X_test = X_test.reshape((-1, CHUNK_SIZE, 1)) 
X_test_scaled = X_test_scaled.reshape((-1, CHUNK_SIZE, 1)) 
y_train = y_train.reshape((-1, OUTPUT_SIZE, 1)) 
y_test = y_train.reshape((-1, OUTPUT_SIZE, 1)) 

print(f"Final    shapes: {X_train.shape}, {X_train_scaled.shape}, {X_test.shape}, {X_test_scaled.shape}, {y_train.shape}, {y_test.shape}")

Original shapes: (287676, 128), (95892, 128), (287676,), (95892,)
Final    shapes: (287676, 128, 1), (287676, 128, 1), (95892, 128, 1), (95892, 128, 1), (287676, 1, 1), (287676, 1, 1)


# Hyperband Tuning

In [9]:
import model_tuner

MAX_EPOCHS_PER_MODEL = 20 # Meant to get a decent idea of parameter, not create a final model. Behaves oddly below 3.
HYPERBAND_ITERATIONS = 2  # "Number of times to iterate over the full Hyperband algorithm"
EXECUTIONS_PER_TRIAL = 2  # Training from scratch
SEARCH_FIT_EPOCHS = 20    # Epochs for each attempt to do a fit, I think. Not sure how this relates to MAX_EPOCHS_PER_MODEL.
OVERWRITE = True          # I'm hoping to be able to interrupt a run and resume it later

input_shape = (CHUNK_SIZE,1)
mr_t = model_tuner.ModelTuner(input_shape, OUTPUT_SIZE, CHUNK_SIZE, BATCH_SIZE)

# All-encompassing optimization parameter choices. Do not try to use all of them at once...
mr_t.CHOICES_PROCESSING_UNITS = [1, CHUNK_SIZE // 16, CHUNK_SIZE // 4, CHUNK_SIZE, CHUNK_SIZE * 2]
mr_t.CHOICES_ACTIVATIONS = ["elu", "gelu", "hard_sigmoid", "hard_silu", "hard_swish", "leaky_relu", "linear", "log_softmax", "mish",
        "relu", "relu6", "selu", "sigmoid", "silu", "softmax", "softplus", "softsign", "swish", "tanh"]
mr_t.CHOICES_FANCY_TOPO = ["GRU", "RNN", "LSTM", "GRU-RNN", "GRU-LSTM", "GRU-RNN-LSTM"]
mr_t.CHOICES_USE_OUTPUT_LIMITER = [True, False] # Prefers True
mr_t.CHOICES_OPTIMIZER = ["adamax", "sgd", "RMSProp"]

# Narrow down the choices as needed.
mr_t.CHOICES_PROCESSING_UNITS = [CHUNK_SIZE]
mr_t.CHOICES_FANCY_TOPO = ["NONE"]
mr_t.CHOICES_USE_OUTPUT_LIMITER = [True]
mr_t.CHOICES_OPTIMIZER = ["adamax"]

if TUNE_QUICKLY:
    MAX_EPOCHS_PER_MODEL = 3
    HYPERBAND_ITERATIONS = 1
    EXECUTIONS_PER_TRIAL = 1
    SEARCH_FIT_EPOCHS = 4

# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    return mr_t.CreateModel(hp)

# Run the kerastuner search for best hyperparameters
if TUNE_NETWORK:
    if USE_CUSTOM_METRICS:
        objective = kt.Objective("val_modulo_distance_accuracy", direction="max")
    else:
        objective = kt.Objective(f"val_{MAIN_ACCURACY_METRIC}", direction="max")

    tuner = kt.Hyperband(
        create_model,
        objective=objective,
        max_epochs=MAX_EPOCHS_PER_MODEL,
        hyperband_iterations=HYPERBAND_ITERATIONS,
        executions_per_trial=EXECUTIONS_PER_TRIAL,
        overwrite=OVERWRITE,
        directory=TUNER_DIRECTORY,
        project_name=TUNER_PROJECT_NAME)
    tuner.search(X_train_scaled, y_train, epochs=SEARCH_FIT_EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test_scaled,y_test))
    
    best_hyper = tuner.get_best_hyperparameters(1)[0]
    print(f"Best Hyper Values: {best_hyper.values}")
    
    nn = tuner.get_best_models(1)[0]
    eval_results = nn.evaluate(X_test_scaled, y_test, verbose=2, batch_size=BATCH_SIZE )
    print(f"Best Model Loss: {eval_results[0]}, Accuracy: {eval_results[1:]}")

    nn.save("./saved_models/tuned.keras")

# Model Reload /Creation

In [10]:
if BUILD_NETWORK:
    print("Building new model")
    nn = tf.keras.models.Sequential()

    input_shape = (None, 1, CHUNK_SIZE)
    input_layer = tf.keras.Input(shape=input_shape[1:], name="Input_Layer")
    nn.add(input_layer)

    activation_A = "tanh"
    recurrent_activation_A = "sigmoid"
    lstm_layer = tf.keras.layers.LSTM(PROCESSING_UNITS, return_sequences=True, activation=activation_A, recurrent_activation=recurrent_activation_A,
        name=f"A_LSTM_{activation_A}_{recurrent_activation_A}")
    nn.add(lstm_layer)

    output_layer = tf.keras.layers.Dense(units = OUTPUT_SIZE, activation=modulo_output, name='Modulo_Layer')
    nn.add(output_layer)

    # Check the structure of the model
    print(f"Input shape: {nn.input_shape}, Output shape: {nn.output_shape}")
    print(nn.summary())

Building new model


I0000 00:00:1733031190.792565   29844 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733031190.936161   29844 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733031190.936210   29844 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733031190.939835   29844 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733031190.939887   29844 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Input shape: (None, 1, 128), Output shape: (None, 1, 1)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ A_LSTM_tanh_sigmoid (LSTM)      │ (None, 1, 32)          │        20,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Modulo_Layer (Dense)            │ (None, 1, 1)           │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,641 (80.63 KB)

 Trainable params: 20,641 (80.63 KB)

 Non-trainable params: 0 (0.00 B)

None


# Model Training

In [ ]:
# Reset the "best" score.
# If you're manually training iteratively, comment this out to preserve the best-ness:
model_checkpoint_callback = initialize_save_best(BEST_PATH)

# It can be helpful to load the best, train some more, and try to improve it:
if LOAD_BEST_MODEL:
    if os.path.exists(BEST_PATH):
        print(f"Loading model from {BEST_PATH}")
        nn = tf.keras.models.load_model(BEST_PATH,
            custom_objects={
                'modulo_distance_loss': modulo_distance_loss,
                'modulo_distance_accuracy': modulo_distance_accuracy,
                'modulo_rounded_accuracy': modulo_rounded_accuracy,
                'modulo_output': modulo_output
        })

# Train the model
if TRAIN_MODEL:
    # Decide what metrics to use
    if USE_CUSTOM_METRICS:
        loss = modulo_distance_loss
        metrics = [modulo_distance_accuracy, modulo_rounded_accuracy]
    else:
        loss = LOSS_METRIC
        metrics = [MAIN_ACCURACY_METRIC]

    print(nn.summary())
    print(f"Training model")
    
    if SAVE_BEST_MODEL:
        callbacks = [model_checkpoint_callback]
    else:
        callbacks = None
    
    # Compile the Sequential model together and customize metrics
    nn.compile(loss=loss, optimizer=OPTIMIZER, metrics=metrics)
    
    # Fit the model to the training data
    tmp_x = X_train_scaled.reshape((-1, CHUNK_SIZE, 1)) 
    tmp_y = y_train.reshape((-1, OUTPUT_SIZE, 1)) 
    print(X_train_scaled.shape, y_train.shape, tmp_x.shape, tmp_y.shape)
    #fit_model = nn.fit(X_train_scaled, y_train, epochs=EPOCHS, callbacks=callbacks, batch_size=BATCH_SIZE)
    fit_model = nn.fit(tmp_x, tmp_y, epochs=EPOCHS, callbacks=callbacks, batch_size=BATCH_SIZE)

print(f"Input shape: {nn.input_shape}, Output shape: {nn.output_shape}")
print(nn.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ A_LSTM_tanh_sigmoid (LSTM)      │ (None, 1, 32)          │        20,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Modulo_Layer (Dense)            │ (None, 1, 1)           │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,641 (80.63 KB)

 Trainable params: 20,641 (80.63 KB)

 Non-trainable params: 0 (0.00 B)

None
Training model
(287676, 128, 1) (287676, 1, 1) (287676, 128, 1) (287676, 1, 1)
Epoch 1/10


2024-11-30 21:33:14.326696: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90101


279/281 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 2.4965 - modulo_distance_accuracy: 0.5007 - modulo_rounded_accuracy: 0.0762
Epoch 1: modulo_distance_accuracy improved from -inf to 0.53208, saving model to ./saved_models/best.keras
281/281 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - loss: 2.4948 - modulo_distance_accuracy: 0.5010 - modulo_rounded_accuracy: 0.0764
Epoch 2/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.7906 - modulo_distance_accuracy: 0.6419 - modulo_rounded_accuracy: 0.1721
Epoch 2: modulo_distance_accuracy improved from 0.53208 to 0.66114, saving model to ./saved_models/best.keras
281/281 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 1.7903 - modulo_distance_accuracy: 0.6419 - modulo_rounded_accuracy: 0.1722
Epoch 3/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.5917 - modulo_distance_accuracy: 0.6817 - modulo_rounded_accuracy: 0.2173
Epoch 3: modulo_distance_accuracy improved from 0.66114 to 0.68502, saving model to ./saved_models/best.keras
281/281 ━━━━━━━━━━━━━

In [ ]:
# Evaluate the model using the test data

# Predicting the whole test set can take a lot of memory, so this can be used to limit it:
TEST_SET_SIZE = X_test_scaled.shape[0]
MAX_TEST_SUBSET = 10000
TEST_SUBSET_SIZE = min(TEST_SET_SIZE, MAX_TEST_SUBSET)
X_test_scaled_subset = X_test_scaled[0:TEST_SUBSET_SIZE, :, :]
y_test_subset = y_test[0:TEST_SUBSET_SIZE, :, :]

if USE_CUSTOM_METRICS:
    print("Evaluating with model.predict() ...")    
    raw_pred = nn.predict(X_test_scaled_subset, batch_size=BATCH_SIZE)
    
    print(f"raw_pred: {raw_pred.shape}")
    if INFER_KEY:
        # Average together all the predictions
        y_pred = np.array([np.median(raw_pred[x,:,:]) for x in range(raw_pred.shape[0])]).astype(np.float64)
    else:
        # This is probably wrong...
        y_pred = raw_pred[:,1,:]
    print(f"y_pred: {y_pred.shape}")

    # My custom loss and accuracy functions are running out of memory for decoded texts for some reason...
    if INFER_KEY:
        loss = modulo_distance_loss(y_test_subset, y_pred)
        accuracy_distance = modulo_distance_accuracy(y_test_subset, y_pred)
        accuracy_rounded = modulo_rounded_accuracy(y_test_subset, y_pred)
        print(f"Loss: {loss:0.6}, Accuracy (Distance): {accuracy_distance:0.6}, Accuracy (Rounded): {accuracy_rounded:0.6}")

    if INFER_KEY:
        print(f"y_pred: {y_pred.shape}, y_test: {y_test_subset.shape}")
        pred_pd = pd.DataFrame(preds)
        true_pd = pd.DataFrame(y_test_subset[:,0,0])
        print("Inferred key distribution:\n", pred_pd.describe())
        print("Inferred key value counts:\n", pred_pd[0].round().value_counts())
        print("True key distribution    :\n", true_pd.describe())
        print("True key value counts    :\n", true_pd[0].round().value_counts())

# I don't think evaluate() is handling the plethora of keys output by the model. These metrics are probably quite wrong:
print("Evaluating with model.evaluate() ...")
eval_results = nn.evaluate(X_test_scaled_subset, y_test_subset, verbose=2, batch_size=BATCH_SIZE)
print(f"PROBABLY INCORRECT: Loss: {eval_results[0]}, Accuracy: {eval_results[1:]}")

# Model Usefulness Spot-Check

In [ ]:
# These only really work for the Caesar Cipher

def decode_chunks_with_model(chunks: list[list], model, scaler, input_already_scaled = True) -> list[list]:
    if input_already_scaled:
        return model.predict(chunks)
    else:
        return model.predict(scaler.transform(chunks))

def decode_text_with_model(ciphertext: str, model, scaler) -> str:
    offset_chunks = helpers.chunkify(encoders.string_to_offsets(ciphertext), CHUNK_SIZE)
    decoded_chunks = decode_chunks_with_model(offset_chunks, model, scaler, input_already_scaled = False)
    rounded = np.rint(decoded_chunks.flatten()).astype(int)
    return encoders.offsets_to_string(rounded)

def infer_key_with_model(ciphertext: str, model, scaler) -> int:
    chunks = helpers.string_to_bytes(ciphertext, CHUNK_SIZE)    
    scaled_chunks = scaler.transform(chunks)
    #keys = model.predict(scaled_chunk)
    tmp_x = np.array(scaled_chunks).reshape((-1, CHUNK_SIZE, 1))
    keys = model.predict(tmp_x)
    key = int(round(np.median(keys)))
    return key

if INFER_TEXT:    
    CHUNKS_TO_CHECK = 2
else:
    CHUNKS_TO_CHECK = 20

cipher_file_db = sid_to_c[list(sid_to_c.keys())[0]][0]
ciphertext_path = cipher_file_db.path
ciphertext = helpers.read_text_file(ciphertext_path)
ciphertext = ciphertext[0:CHUNK_SIZE * CHUNKS_TO_CHECK]

if INFER_TEXT:    
    print("Decoded   : ", decode_text_with_model(ciphertext, nn, X_scaler))
if INFER_KEY:
    with db.get_session() as session:
        correct_key = db.get_key_by_id(session, cipher_file_db.key_id).value
    print("Correct Key: ", correct_key)
    
    inferred_key = infer_key_with_model(ciphertext, nn, X_scaler)
    print("Inferred Key: ", inferred_key)
    
    chunks = helpers.string_to_bytes(ciphertext, CHUNK_SIZE)
    scaled_chunks = scaler.transform(chunks)
    pred = nn.predict(scaled_chunks)
    
    #tmp_x = np.array(scaled_chunks).reshape((-1, CHUNK_SIZE, 1))
    #pred = nn.predict(tmp_x)

    #print(pred)
    #print(pd.DataFrame(pred).describe())
